In [3]:
from corus import load_lenta
import pandas as pd

path = 'C:\VsCodeProjects\hackaton-nlu-model\data\lenta-ru-news.csv.gz'
records = load_lenta(path)


In [5]:
dataset = pd.DataFrame(rec.text for rec in records)

In [6]:
data = dataset.iloc[:500, :]

In [7]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

sentences = data[0].to_list()

embeddings_list = []

for s in sentences:
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = model_output.pooler_output
    embeddings_list.append((embedding)[0].numpy())

embeddings = np.asarray(embeddings_list)

embeddings.shape

c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(500, 768)

In [8]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

def determine_k(embeddings):
    k_min = 10
    clusters = [x for x in range(2, k_min * 11)]
    metrics = []
    for i in clusters:
        metrics.append((KMeans(n_clusters=i).fit(embeddings)).inertia_)
    k = elbow(k_min, clusters, metrics)
    return k

def elbow(k_min, clusters, metrics):
    score = []

    for i in range(k_min, clusters[-3]):
        y1 = np.array(metrics)[:i + 1]
        y2 = np.array(metrics)[i:]
    
        df1 = pd.DataFrame({'x': clusters[:i + 1], 'y': y1})
        df2 = pd.DataFrame({'x': clusters[i:], 'y': y2})
    
        reg1 = LinearRegression().fit(np.asarray(df1.x).reshape(-1, 1), df1.y)
        reg2 = LinearRegression().fit(np.asarray(df2.x).reshape(-1, 1), df2.y)

        y1_pred = reg1.predict(np.asarray(df1.x).reshape(-1, 1))
        y2_pred = reg2.predict(np.asarray(df2.x).reshape(-1, 1))    
        
        score.append(mean_squared_error(y1, y1_pred) + mean_squared_error(y2, y2_pred))

    return np.argmin(score) + k_min

k = determine_k(embeddings)

print(k)

c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\clus

15


In [9]:
kmeans = KMeans(n_clusters = k, random_state = 42).fit(embeddings)
kmeans_labels = kmeans.labels_

data = pd.DataFrame()
data['text'] = sentences
data['label'] = kmeans_labels

c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [10]:

for t in data[data['label'] == 0]['text'].head(28):
    print(t)

Подразделения Народной милиции самопровозглашенной Донецкой народной республики (ДНР) приведены в полную боевую готовность для отражения наступления Вооруженных сил Украины (ВСУ). Об этом в пятницу, 14 декабря, заявил заместитель командующего оперативным командованием ДНР Эдуард Басурин, передает Донецкое агентство новостей. «С момента получения нами информации о подготовке широкомасштабного наступления со стороны Украины наши подразделения доукомплектованы личным составом, обеспечены необходимым вооружением, техникой и боеприпасами», — указал Басурин. По его словам, на занимаемых военнослужащими республики позициях провели инженерные работы, а на танкоопасных направлениях установили противотанковые заграждения. Кроме того, подразделения Народной милиции были обеспечены необходимым количеством противотанковых ракетных комплексов, позволяющих сдержать до двух танковых бригад ВСУ. Ключевые объекты усилены средствами противовоздушной обороны. 13 декабря о готовящемся наступлении ВСУ на ДН

In [11]:
for t in data[data['label'] == 1]['text'].head(28):
    print(t)

Компания Johnson&Johnson, уличенная в использовании асбеста в производстве детской присыпки, с 1970 годов скрывала от регулирующих органов информацию о наличии канцерогена в своих продуктах. Об этом сообщает Reuters. Тесты, проведенные в начале 1970-х, подтвердили присутствие вредного для здоровья вещества в присыпке, причем в некоторых случаях «в очень высокой концентрации». В то же время расследование, проведенное агентством, показало, что первые следы наличия асбеста в продукции J&J были обнаружены еще в 1957-1958 годах. Johnson&Johnson находили асбест в присыпке Johnson's Baby Powder вплоть до начала 2000-х. При этом компания все это время с помощью подлогов скрывала результаты исследований от Управления по санитарному надзору за качеством пищевых продуктов и медикаментов (FDA). Присыпка, в настоящее время производимая J&J, может быть безопасной. В свою очередь, в компании выступили с опровержением результатов расследования, проведенного Reuters. Тем не менее рыночная стоимость кор

In [12]:
for t in data[data['label'] == 2]['text'].head(28):
    print(t)

Жители Великобритании будут платить семь евро за посещение стран Евросоюза после Brexit. Об этом сообщает BBC News. Въезд нужно будет оплачивать каждые три года. Британцы по-прежнему смогут путешествовать по территории ЕС без визы, однако им придется подавать электронную заявку в системе ETIAS (European Travel Information and Authorization System). До 2021 года будет длиться переходный период, во время которого граждан Британии продолжат пускать в ЕС по паспорту, после этого электронная заявка станет обязательной. Система ETIAS будет действовать для граждан из 61 страны, которые могут находиться в странах Шенгенского соглашения без визы до 90 дней. Евросоюз принял решение ужесточить пограничный контроль в связи с миграционным кризисом и усилением террористической угрозы. Предполагается, что 95 процентов заявок будут получать одобрение в течение нескольких минут, однако рассмотрение заявления может занять до четырех недель, также его могут отклонить. В случае положительного решения разр

In [13]:
for t in data[data['label'] == 3]['text'].head(28):
    print(t)

Российская молодежь лучше усвоит духовные ценности, если распространять их через интернет и социальные сети. С такой инициативой выступил глава Федерального агентства по делам молодежи Александр Бугаев в ходе религиозного форума «Россия в будущее», передает радио «Говорит Москва». Бугаев отметил, что раньше с этой функцией справлялись плакаты, которые можно было повесить в учебных заведениях, однако в настоящее время этого недостаточно. «Намного эффективнее будет, если мы об этом хорошем, добром и ценном расскажем в социальных группах в интернете. При помощи мессенджеров, неформальных объединений молодежи, которые зачастую охватывают огромное количество людей», — сказал он. По словам чиновника, современные способы распространения информации помогут молодым людям лучше уяснить моральные ценности. В мае стало известно, что в российских школах могут начать преподавать новый курс для учеников и их родителей — о семейном укладе. В рамках курса дети будут изучать основы семейной психологии, 

In [14]:
for t in data[data['label'] == 4]['text'].head(28):
    print(t)

 Капитан «Вашингтона» Александр Овечкин сделал хет-трик в игре Национальной хоккейной лиги (НХЛ) с «Каролиной». Таким образом он продлил результативную серию до 13 матчей и повторил личный рекорд января-февраля 2007 года. Об этом сообщает Sports.ru. Встреча завершилась со счетом 5:6 Б — «Вашингтон» переиграл «Каролину» по булитам. Для россиянина это второй подряд хет-трик. До этого он отметился им в матче против «Детройт Ред Уингс». Клубный рекорд остается за Майклом Гартнером — 17 матчей. В соперничестве бомбардиров Овечкин занял пятое место с 42 (28+14) очками в 31 матче.
Нападающий клуба «Эдмонтон Ойлерс» Коннор Макдэвид забросил шайбу с отрицательного угла в матче регулярного чемпионата Национальной хоккейной лиги (НХЛ) с «Филадельфией Флайерс». Видео появилось в Twitter-аккаунте команды. Форвард вынудил игрока соперника ошибиться, а затем добил шайбу с отрицательного угла (из-за ворот). «Эдмонтон Ойлерс» обыграл «Филадельфию Флайерс» со счетом 4:1. В активе Макдэвида 48 (19+29) оч

In [15]:
for t in data[data['label'] == 5]['text'].head(28):
    print(t)

Вице-президент Еврокомиссии Марош Шефчович пригласил Россию и Украину в Брюссель для следующего этапа трехсторонних переговоров по газу. Встреча состоится в январе, написал он в Twitter. «Ожидаем, что коммерческие структуры также примут участие. Важно развивать положительную динамику», — отметил Шефчович. Ранее глава Минэнерго России Александр Новак сообщал, что переговоры, возможно, пройдут во второй половине января, добавляет РИА Новости. 13 декабря стало известно, что «Нафтогаз Украины» обратился в американские суды с исками о принудительном взыскании долга с российского «Газпрома». «Нафтогаз» пытается получить от российской стороны 2,7 миллиарда долларов — именно столько «Газпром» должен выплатить Украине по решению Стокгольмского арбитража за недопоставку газа по транзиту.
Президент Украины Петр Порошенко занял место в президиуме объединительного собора церковных структур страны, который в субботу, 15 декабря, проходит в Киеве. Фотографию опубликовал пресс-секретарь главы государс

In [16]:
for t in data[data['label'] == 6]['text'].head(28):
    print(t)

В отношении чиновников департамента градостроительства мэрии Красноярска возбуждено уголовное дело о халатности в особо крупном размере. Об этом в пятницу, 14 декабря сообщается на сайте краевого управления Следственного комитета России (СКР). По данным следствия, в период с 2017 по 2018 год неустановленные госслужащие приобретали квартиры для детей-сирот по завышенной цене, нарушая при этом порядок проведения аукционов и производя ошибочный расчет цены за квадратный метр. Таким образом им удалось закупить около 360 квартир, а ущерб от их махинаций составил более 61 миллиона рублей. В 2017 году следователи передали в суд уголовное дело бывшего начальника управления жилищной политики администрации города Оренбурга Сергея Макеева, обвиняемого в превышении должностных полномочий. По версии следствия, в период с октября 2015 года по 31 декабря 2015 года чиновник подписал акты о приеме 94 квартир в новостройках, которые предназначались детям-сиротам и жильцам аварийных домов.
В Пскове в отн

In [17]:
for t in data[data['label'] == 7]['text'].head(28):
    print(t)

Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Хохфильцене вечером 12 декабря. Как написал биатлонист Александр Логинов, их считают виновными в махинациях с переливанием крови. Биатлонисту Антону Шипулину, также попавшему в список, полиция нанесла отдельный визит: сейчас он тренируется отдельно в австрийском Обертиллахе. Обвинения спортсмен назвал бредом, а также указал на «охоту на ведьм» в мировом биатлоне. В Австрии прием допинга — уголовное пре

In [112]:
for t in data[data['label'] == 8]['text'].head(28):
    print(t)

Полет российских бомбардировщиков над Карибским морем сняли на видео
Бутину допросят в США по новому уголовному делу
Пентагон проигнорировал обращение России по ракетному договору
Су-27 разбился на Украине
Финансовой мощи США предсказали ослабление
В США объяснили провал российского «Панциря» в Сирии
Российские Ту-160 в Венесуэле назвали «музейными экспонатами»
Россия рассказала о судьбе ракетного договора с США
В Минобороны назвали дилетантскими заявления США о Ту-160
МС-21 и Sukhoi Superjet 100 получат «революционные уши»
Армию ДНР привели в полную боевую готовность
Сенат США потребовал прекратить военную помощь саудитам
Европа отказалась запускать спутники с Восточного
Лукашенко объяснил поставки оружия Азербайджану
Назван победитель боя между Су-57 и F-35
Американский истребитель упал в океан
Россия напомнила Японии о выводе американских военных
Полет российских Ту-160 над Карибским морем показали на видео
Стивен Сигал оценил отношения России и США
Россия создаст двигатель «напарни

In [113]:
for t in data[data['label'] == 9]['text'].head(28):
    print(t)

Путин предостерег от запретов рэп-концертов
Путин подарил тяжелобольному мальчику обещанный полет на вертолете
Порошенко уселся в президиуме собора украинских церквей
Путин порассуждал о сексе, наркотиках и протесте в рэпе
Кот Путина освоил японский язык
Лавров рассказал о пытках Бутиной в США
«Яндекс» определил дату смерти Петра Порошенко
Песков ответил на вопрос о признании Бутиной
В «Яндексе» объяснили внезапную «кончину» Порошенко
Лукашенко пожаловался Путину на украинских «отмороженных нацменов»
Лукашенко обиделся на Россию из-за газового обмана
Лукашенко рассказал свою правду об извинениях перед Путиным
Стала известна дата встречи Путина и Лукашенко
Лукашенко распек Пашиняна и рассказал о позоре
Задержан вице-губернатор Псковской области
Владимир Кличко захотел вернуться на ринг
Лукашенко порассуждал о мате и вспомнил советы Ельцина
Порошенко обозначил следующий шаг противодействия России
Лукашенко объяснил ненужность российской авиабазы в Белоруссии
Путин признался в большом инт

In [114]:
for t in data[data['label'] == 10]['text'].head(28):
    print(t)

Хакеры рассказали о планах Великобритании заминировать Севастополь
В Минобороны раскрыли многомиллионные хищения при ремонте зениток
Овечкин повторил свой рекорд
Хоккеист НХЛ забросил шайбу с отрицательного угла
Испанские клубы открестились от Неймара
Неизвестные захватили собор канонической УПЦ
Падение самосвала на аквапарк в Ялте сняли на видео
Моуринью сравнил футболистов с мебелью
Падение горнолыжника на полной скорости попало на видео
Путин пообещал поразбираться с Cirque du Soleil в Сколково
Умер актер «Кривого зеркала»
Фанат «Спартака» решил обвинить Глушакова в побоях
Крещение ребенка грубой силой попало на видео
Иглесиас и Курникова оставят родственников без особняка
За сравнившего с быдлом зрителей актера «Универа» заступились
Названы отличия качественного игристого вина
В Windows 10 нашли ад перфекциониста
Седина станет модной
Украинский десантник объяснился за шеврон СС и попал впросак
Аршавина вернут в «Зенит»
Умерла отсудившая у Клинта Иствуда миллионы актриса
Умерла певи

In [115]:
for t in data[data['label'] == 11]['text'].head(28):
    print(t)

Вандалы потушили старейший в России Вечный огонь
В Петербурге нашли квартиру из 58 комнат
Главная реформа Обамы признана неконституционной
Стали известны все участники плей-офф Лиги Европы
В Антарктиде нашли мертвыми двух техников
Предсказана глобальная нехватка питьевой воды
Названа новая опасность секс-роботов будущего
Ирландия стала на шаг ближе к легальным абортам
Раскрыта новая причина старения
Обнаружена быстро погибающая планета
Раскрыта реальная внешность модели Victoria’s Secret
Обнаружено письмо девочки Санте 120-летней давности
Мэй осталась премьер-министром Великобритании
Определились все участники плей-офф Лиги чемпионов
Названа неожиданная опасность уборки пыли
Назван главный след людей в истории Земли
В городах заметили распространение опасных мертвых зон
Обнаружена новая мощная атака на ядерную и финансовую структуры
Дальний Восток обрел новую столицу
Марадону выгнали из дома
Определились первые участники плей-офф Лиги чемпионов
Кофе оказался средством против старения м

In [116]:
for t in data[data['label'] == 12]['text'].head(28):
    print(t)

Британцам придется заплатить за путешествия по Европе
Пассажир московского метро заплатит 20 тысяч рублей за отказ от досмотра
В Еврейской автономной области китаец лишился шишек на три миллиона
Тысячи долларов выпали из машины инкассаторов и привели к коллапсу
Телефонную книгу Сталина продали за три миллиона рублей
Хакеры пригрозили американцам бомбами и потребовали денег
Шнуров рассказал о заработанном в Мариинском театре миллионе
Выплативший 13 миллионов штрафа Роналду снова оказался в суде
Россиянин получил 30 тысяч рублей за пытки в полиции
Миллиардную реформу признали провальной
Российские нефтяники заработали триллионы на фоне бензинового кризиса
Россияне собирали деньги для ИГ под предлогом помощи жителям Африки
Миллионер сознался в убийстве любовницы во время секса
Депутаты распределили между собой 356 миллионов рублей и пожалели
В ФАС провели обыски по делу о мошенничестве на два миллиона долларов
Китаец застраховал жену на миллионы и утопил
Росгвардия потратит 200 миллионов 

In [117]:
for t in data[data['label'] == 13]['text'].head(28):
    print(t)

Названы регионы России с самой высокой смертностью от рака
Обнаружено самое счастливое место на планете
Названы регионы России с самым дорогим и дешевым новогодним столом
Названы районы Петербурга с самым дешевым жильем
В сети разоблачили самые популярные страхи жителей планеты
Найдена самая дорогая однокомнатная квартира России
Названа главная законодательница моды последних ста лет
Названы лучшие фильмы года
Россияне определились с главным событием и человеком года
Названы самые ненадежные пароли года
Названы самые привлекательные для воров вещи туристов
Названо лучшее сливочное масло
Назван «Джентльмен года» в российском футболе
Google выбрал своего «Человека года»
Про самого красивого американского маньяка сделают сериал
Названы главные мемы года
Названы самые стильные люди года
Названы главные страхи владельцев элитного жилья
Названа самая востребованная еда в мире
Названа самая прослушиваемая песня XX века
Определены регионы с самым большим числом должников по ЖКХ
Названы лучшие 